## feature extraction

brain 폴더 안에 각 환자별로 brainimage가 있고, mask 폴더 안에 각 환자별로 mask가 있다. \
brain image m개와 mask n개를 조합하여 각 brain_image-mask쌍에 대해 feature extraction하기 => itertools의 product이용

feature를 extract하여 각 환자가 row에 각 feature가 column에 오도록 csv파일로 뽑아내기

In [1]:
!pip install SimpleITK
!pip install pyradiomics


In [12]:
from glob import glob
import SimpleITK as sitk
import os
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
from shutil import move
from itertools import product
import pandas as pd

In [13]:
PATH = '/media/sc-mlsg/PYW98854602/20210507_G3_Glioma_autoseg_preprocessed/'
in_path = PATH + '05_Stripped_znormalized/'
outpath = PATH
maskDir = PATH + '00_Tumor_masks_resampled/'

In [14]:
for (path, dirs, files) in os.walk(in_path):
    print(path, dirs, files)

/media/sc-mlsg/PYW98854602/20210507_G3_Glioma_autoseg_preprocessed/05_Stripped_znormalized/ ['5992119', '7630601', '2659261', '2917442', '3173505', '3238199', '4240406', '4334564', '4413737', '5101886', '5107993', '5199303', '5222508', '5319033', '5390880', '5408861', '5533514', '5643165', '5659614', '5705253', '5820198', '5984320', '6594875', '6597378', '6776630', '7031484', '7368360', '7373066', '7415095', '7432750', '7439815', '7459615', '7470864', '7478649', '7494459', '7513852', '7522152', '7525220', '7540951', '7550338', '7560945', '7606928', '7654630', '7664524', '7677983', '7687513', '7711873', '7716992', '7732014', '7741219', '7753977', '7760652', '7767219', '7781243', '7783083', '7809569', '7816559', '7820534', '7821391', '7842502', '7871704', '7892450', '7898823', '7922825', '7946503', '7955082', '8011880', '8037487', '8064434', '8074420', '8082756', '8086467', '8089857', '8164554', '8170673'] []
/media/sc-mlsg/PYW98854602/20210507_G3_Glioma_autoseg_preprocessed/05_Stripped_

In [15]:
from __future__ import print_function
from radiomics import featureextractor
import radiomics
import logging
radiomics.logger.setLevel(logging.ERROR)
import SimpleITK as sitk
from radiomics import firstorder, glcm, shape, glrlm, glszm, ngtdm, gldm
import numpy as np


def Numpy2Itk(array):
    '''
    :param array: numpy array format
    :return: simple itk image type format
    '''
    return sitk.GetImageFromArray(array)

def feature_extract(image_origin, image_mask, features = ['firstorder', 'glcm', 'glszm', 'glrlm', 'ngtdm', 'shape'], binWidth=32, binCount=None):
    '''
    :param image_origin: image_array (numpy array)
    :param image_mask: mask_array (numpy array)
    :return: whole features, featureVector
    '''
    image = Numpy2Itk(image_origin)
    mask = Numpy2Itk(image_mask)
    
    settings = {}

    if binWidth:
        settings['binWidth'] = binWidth
    if binCount:
        settings['binCount'] = binCount
    settings['resampledPixelSpacing'] = (1,1,1)
    settings['interpolator'] = 'sitkBSpline'
    settings['verbose'] = True
    
    extractor = featureextractor.RadiomicsFeatureExtractor(**settings)
    extractor.settings['enableCExtensions'] = True
    
    for feature in features:
        extractor.enableFeatureClassByName(feature.lower())
        
    featureVector = extractor.execute(image, mask)
    
    cols = []; feats = []
    for feature in features:
        for featureName in sorted(featureVector.keys()):
            if feature in featureName:
                cols.append(featureName)
                feats.append(featureVector[featureName])
    return feats, cols

In [17]:
error_dict = {}
for size in [8, 32, 128]:
    feat_df_width = pd.DataFrame()
    feat_df_count = pd.DataFrame()
    total = 0
    for (path, dirs, files) in os.walk(in_path):
        if not files:
            continue
        total += 1
        pt = path.split(os.path.sep)[-1]
        print(total, pt)
        brain_paths = sorted(glob(os.path.join(path, '*')))
#        mask_paths = glob(os.path.join(maskDir, pt, '*ce*')) + glob(os.path.join(maskDir, pt, '*peri*'))
        mask_paths = sorted(glob(os.path.join(maskDir, pt, '*')))
        if not mask_paths:
            print("tumor mask not exists")
            continue
        bw_features_lst, bw_cols_lst, bc_features_lst, bc_cols_lst = [], [], [], []
        
        for (img_path, mask_path) in product(brain_paths, mask_paths):
            try:
                seq = os.path.basename(img_path).split("_")[1]
    #                if seq in ["F", "T1", "T2F"]:
    #                    continue
                print(img_path, mask_path)
                mask_type = os.path.basename(mask_path).split("_")[0]

                brain_img, mask = [sitk.ReadImage(path) for path in [img_path, mask_path]]
                brain_arr, mask_arr = [sitk.GetArrayFromImage(img) for img in [brain_img, mask]]

                # Scale
                brain_arr = brain_arr * 100

                # Change binWidth
                bw_features, bw_cols = feature_extract(brain_arr, mask_arr, binWidth=size)
                bw_cols = [col.replace('original', '{}'.format(seq)) for col in bw_cols]
#                bw_cols = [col.replace('original', '{}_image_{}_mask'.format(seq, mask_type)) for col in bw_cols]

                # Change binCount            
                bc_features, bc_cols = feature_extract(brain_arr, mask_arr, binWidth=None, binCount=size)
                bc_cols = [col.replace('original', '{}'.format(seq)) for col in bc_cols]
#                bc_cols = [col.replace('original', '{}_image_{}_mask'.format(seq, mask_type)) for col in bc_cols]

                bw_features_lst += bw_features
                bw_cols_lst += bw_cols
                bc_features_lst += bc_features
                bc_cols_lst += bc_cols

            except Exception as e:
                print("error", e, path)
                error_dict[path] = e
                
        feat_df_width = pd.concat([feat_df_width, pd.DataFrame(np.array(bw_features_lst),index=bw_cols_lst, columns=[pt]).T])
        feat_df_count = pd.concat([feat_df_count, pd.DataFrame(np.array(bc_features_lst), index=bc_cols_lst, columns=[pt]).T])

    feat_df_width.to_csv(os.path.join(outpath, '{}_bin_width_{}_scale_100.csv'.format('G3_Glioma', size)))
    feat_df_count.to_csv(os.path.join(outpath, '{}_bin_count_{}_scale_100.csv'.format('G3_Glioma', size)))
    print("Finished %d size" % size)

1 5992119
/media/sc-mlsg/PYW98854602/20210507_G3_Glioma_autoseg_preprocessed/05_Stripped_znormalized/5992119/CT1_r2s_bet_reg_znormed.nii.gz /media/sc-mlsg/PYW98854602/20210507_G3_Glioma_autoseg_preprocessed/00_Tumor_masks_resampled/5992119/5992119_mask.nii.gz
/media/sc-mlsg/PYW98854602/20210507_G3_Glioma_autoseg_preprocessed/05_Stripped_znormalized/5992119/FLAIR_r2s_bet_reg_znormed.nii.gz /media/sc-mlsg/PYW98854602/20210507_G3_Glioma_autoseg_preprocessed/00_Tumor_masks_resampled/5992119/5992119_mask.nii.gz
/media/sc-mlsg/PYW98854602/20210507_G3_Glioma_autoseg_preprocessed/05_Stripped_znormalized/5992119/T1_r2s_bet_reg_znormed.nii.gz /media/sc-mlsg/PYW98854602/20210507_G3_Glioma_autoseg_preprocessed/00_Tumor_masks_resampled/5992119/5992119_mask.nii.gz
/media/sc-mlsg/PYW98854602/20210507_G3_Glioma_autoseg_preprocessed/05_Stripped_znormalized/5992119/T2_r2s_bet_reg_znormed.nii.gz /media/sc-mlsg/PYW98854602/20210507_G3_Glioma_autoseg_preprocessed/00_Tumor_masks_resampled/5992119/5992119_ma

/media/sc-mlsg/PYW98854602/20210507_G3_Glioma_autoseg_preprocessed/05_Stripped_znormalized/4413737/FLAIR_r2s_bet_reg_znormed.nii.gz /media/sc-mlsg/PYW98854602/20210507_G3_Glioma_autoseg_preprocessed/00_Tumor_masks_resampled/4413737/4413737_mask.nii.gz
/media/sc-mlsg/PYW98854602/20210507_G3_Glioma_autoseg_preprocessed/05_Stripped_znormalized/4413737/T1_r2s_bet_reg_znormed.nii.gz /media/sc-mlsg/PYW98854602/20210507_G3_Glioma_autoseg_preprocessed/00_Tumor_masks_resampled/4413737/4413737_mask.nii.gz
/media/sc-mlsg/PYW98854602/20210507_G3_Glioma_autoseg_preprocessed/05_Stripped_znormalized/4413737/T2_r2s_bet_reg_znormed.nii.gz /media/sc-mlsg/PYW98854602/20210507_G3_Glioma_autoseg_preprocessed/00_Tumor_masks_resampled/4413737/4413737_mask.nii.gz
10 5101886
/media/sc-mlsg/PYW98854602/20210507_G3_Glioma_autoseg_preprocessed/05_Stripped_znormalized/5101886/CT1_r2s_bet_reg_znormed.nii.gz /media/sc-mlsg/PYW98854602/20210507_G3_Glioma_autoseg_preprocessed/00_Tumor_masks_resampled/5101886/5101886_m

/media/sc-mlsg/PYW98854602/20210507_G3_Glioma_autoseg_preprocessed/05_Stripped_znormalized/5533514/T1_r2s_bet_reg_znormed.nii.gz /media/sc-mlsg/PYW98854602/20210507_G3_Glioma_autoseg_preprocessed/00_Tumor_masks_resampled/5533514/5533514_mask.nii.gz
/media/sc-mlsg/PYW98854602/20210507_G3_Glioma_autoseg_preprocessed/05_Stripped_znormalized/5533514/T2_r2s_bet_reg_znormed.nii.gz /media/sc-mlsg/PYW98854602/20210507_G3_Glioma_autoseg_preprocessed/00_Tumor_masks_resampled/5533514/5533514_mask.nii.gz
18 5643165
/media/sc-mlsg/PYW98854602/20210507_G3_Glioma_autoseg_preprocessed/05_Stripped_znormalized/5643165/CT1_r2s_bet_reg_znormed.nii.gz /media/sc-mlsg/PYW98854602/20210507_G3_Glioma_autoseg_preprocessed/00_Tumor_masks_resampled/5643165/5643165_mask.nii.gz
/media/sc-mlsg/PYW98854602/20210507_G3_Glioma_autoseg_preprocessed/05_Stripped_znormalized/5643165/FLAIR_r2s_bet_reg_znormed.nii.gz /media/sc-mlsg/PYW98854602/20210507_G3_Glioma_autoseg_preprocessed/00_Tumor_masks_resampled/5643165/5643165_m

/media/sc-mlsg/PYW98854602/20210507_G3_Glioma_autoseg_preprocessed/05_Stripped_znormalized/6776630/T2_r2s_bet_reg_znormed.nii.gz /media/sc-mlsg/PYW98854602/20210507_G3_Glioma_autoseg_preprocessed/00_Tumor_masks_resampled/6776630/6776630_mask.nii.gz
26 7031484
/media/sc-mlsg/PYW98854602/20210507_G3_Glioma_autoseg_preprocessed/05_Stripped_znormalized/7031484/CT1_r2s_bet_reg_znormed.nii.gz /media/sc-mlsg/PYW98854602/20210507_G3_Glioma_autoseg_preprocessed/00_Tumor_masks_resampled/7031484/7031484_mask.nii.gz
/media/sc-mlsg/PYW98854602/20210507_G3_Glioma_autoseg_preprocessed/05_Stripped_znormalized/7031484/FLAIR_r2s_bet_reg_znormed.nii.gz /media/sc-mlsg/PYW98854602/20210507_G3_Glioma_autoseg_preprocessed/00_Tumor_masks_resampled/7031484/7031484_mask.nii.gz
/media/sc-mlsg/PYW98854602/20210507_G3_Glioma_autoseg_preprocessed/05_Stripped_znormalized/7031484/T1_r2s_bet_reg_znormed.nii.gz /media/sc-mlsg/PYW98854602/20210507_G3_Glioma_autoseg_preprocessed/00_Tumor_masks_resampled/7031484/7031484_m

34 7478649
/media/sc-mlsg/PYW98854602/20210507_G3_Glioma_autoseg_preprocessed/05_Stripped_znormalized/7478649/CT1_r2s_bet_reg_znormed.nii.gz /media/sc-mlsg/PYW98854602/20210507_G3_Glioma_autoseg_preprocessed/00_Tumor_masks_resampled/7478649/7478649_mask.nii.gz
/media/sc-mlsg/PYW98854602/20210507_G3_Glioma_autoseg_preprocessed/05_Stripped_znormalized/7478649/FLAIR_r2s_bet_reg_znormed.nii.gz /media/sc-mlsg/PYW98854602/20210507_G3_Glioma_autoseg_preprocessed/00_Tumor_masks_resampled/7478649/7478649_mask.nii.gz
/media/sc-mlsg/PYW98854602/20210507_G3_Glioma_autoseg_preprocessed/05_Stripped_znormalized/7478649/T1_r2s_bet_reg_znormed.nii.gz /media/sc-mlsg/PYW98854602/20210507_G3_Glioma_autoseg_preprocessed/00_Tumor_masks_resampled/7478649/7478649_mask.nii.gz
/media/sc-mlsg/PYW98854602/20210507_G3_Glioma_autoseg_preprocessed/05_Stripped_znormalized/7478649/T2_r2s_bet_reg_znormed.nii.gz /media/sc-mlsg/PYW98854602/20210507_G3_Glioma_autoseg_preprocessed/00_Tumor_masks_resampled/7478649/7478649_m

...

/media/sc-mlsg/PYW98854602/20210507_G3_Glioma_autoseg_preprocessed/05_Stripped_znormalized/7550338/T2_r2s_bet_reg_znormed.nii.gz /media/sc-mlsg/PYW98854602/20210507_G3_Glioma_autoseg_preprocessed/00_Tumor_masks_resampled/7550338/7550338_mask.nii.gz
41 7560945
/media/sc-mlsg/PYW98854602/20210507_G3_Glioma_autoseg_preprocessed/05_Stripped_znormalized/7560945/CT1_r2s_bet_reg_znormed.nii.gz /media/sc-mlsg/PYW98854602/20210507_G3_Glioma_autoseg_preprocessed/00_Tumor_masks_resampled/7560945/7560945_mask.nii.gz
/media/sc-mlsg/PYW98854602/20210507_G3_Glioma_autoseg_preprocessed/05_Stripped_znormalized/7560945/FLAIR_r2s_bet_reg_znormed.nii.gz /media/sc-mlsg/PYW98854602/20210507_G3_Glioma_autoseg_preprocessed/00_Tumor_masks_resampled/7560945/7560945_mask.nii.gz
/media/sc-mlsg/PYW98854602/20210507_G3_Glioma_autoseg_preprocessed/05_Stripped_znormalized/7560945/T1_r2s_bet_reg_znormed.nii.gz /media/sc-mlsg/PYW98854602/20210507_G3_Glioma_autoseg_preprocessed/00_Tumor_masks_resampled/7560945/7560945_m

49 7732014
/media/sc-mlsg/PYW98854602/20210507_G3_Glioma_autoseg_preprocessed/05_Stripped_znormalized/7732014/CT1_r2s_bet_reg_znormed.nii.gz /media/sc-mlsg/PYW98854602/20210507_G3_Glioma_autoseg_preprocessed/00_Tumor_masks_resampled/7732014/7732014_mask.nii.gz
/media/sc-mlsg/PYW98854602/20210507_G3_Glioma_autoseg_preprocessed/05_Stripped_znormalized/7732014/FLAIR_r2s_bet_reg_znormed.nii.gz /media/sc-mlsg/PYW98854602/20210507_G3_Glioma_autoseg_preprocessed/00_Tumor_masks_resampled/7732014/7732014_mask.nii.gz
/media/sc-mlsg/PYW98854602/20210507_G3_Glioma_autoseg_preprocessed/05_Stripped_znormalized/7732014/T1_r2s_bet_reg_znormed.nii.gz /media/sc-mlsg/PYW98854602/20210507_G3_Glioma_autoseg_preprocessed/00_Tumor_masks_resampled/7732014/7732014_mask.nii.gz
/media/sc-mlsg/PYW98854602/20210507_G3_Glioma_autoseg_preprocessed/05_Stripped_znormalized/7732014/T2_r2s_bet_reg_znormed.nii.gz /media/sc-mlsg/PYW98854602/20210507_G3_Glioma_autoseg_preprocessed/00_Tumor_masks_resampled/7732014/7732014_m

/media/sc-mlsg/PYW98854602/20210507_G3_Glioma_autoseg_preprocessed/05_Stripped_znormalized/7816559/FLAIR_r2s_bet_reg_znormed.nii.gz /media/sc-mlsg/PYW98854602/20210507_G3_Glioma_autoseg_preprocessed/00_Tumor_masks_resampled/7816559/7816559_mask.nii.gz
/media/sc-mlsg/PYW98854602/20210507_G3_Glioma_autoseg_preprocessed/05_Stripped_znormalized/7816559/T1_r2s_bet_reg_znormed.nii.gz /media/sc-mlsg/PYW98854602/20210507_G3_Glioma_autoseg_preprocessed/00_Tumor_masks_resampled/7816559/7816559_mask.nii.gz
/media/sc-mlsg/PYW98854602/20210507_G3_Glioma_autoseg_preprocessed/05_Stripped_znormalized/7816559/T2_r2s_bet_reg_znormed.nii.gz /media/sc-mlsg/PYW98854602/20210507_G3_Glioma_autoseg_preprocessed/00_Tumor_masks_resampled/7816559/7816559_mask.nii.gz
58 7820534
/media/sc-mlsg/PYW98854602/20210507_G3_Glioma_autoseg_preprocessed/05_Stripped_znormalized/7820534/CT1_r2s_bet_reg_znormed.nii.gz /media/sc-mlsg/PYW98854602/20210507_G3_Glioma_autoseg_preprocessed/00_Tumor_masks_resampled/7820534/7820534_m

/media/sc-mlsg/PYW98854602/20210507_G3_Glioma_autoseg_preprocessed/05_Stripped_znormalized/7946503/T1_r2s_bet_reg_znormed.nii.gz /media/sc-mlsg/PYW98854602/20210507_G3_Glioma_autoseg_preprocessed/00_Tumor_masks_resampled/7946503/7946503_mask.nii.gz
/media/sc-mlsg/PYW98854602/20210507_G3_Glioma_autoseg_preprocessed/05_Stripped_znormalized/7946503/T2_r2s_bet_reg_znormed.nii.gz /media/sc-mlsg/PYW98854602/20210507_G3_Glioma_autoseg_preprocessed/00_Tumor_masks_resampled/7946503/7946503_mask.nii.gz
66 7955082
/media/sc-mlsg/PYW98854602/20210507_G3_Glioma_autoseg_preprocessed/05_Stripped_znormalized/7955082/CT1_r2s_bet_reg_znormed.nii.gz /media/sc-mlsg/PYW98854602/20210507_G3_Glioma_autoseg_preprocessed/00_Tumor_masks_resampled/7955082/7955082_mask.nii.gz
/media/sc-mlsg/PYW98854602/20210507_G3_Glioma_autoseg_preprocessed/05_Stripped_znormalized/7955082/FLAIR_r2s_bet_reg_znormed.nii.gz /media/sc-mlsg/PYW98854602/20210507_G3_Glioma_autoseg_preprocessed/00_Tumor_masks_resampled/7955082/7955082_m

/media/sc-mlsg/PYW98854602/20210507_G3_Glioma_autoseg_preprocessed/05_Stripped_znormalized/8089857/T2_r2s_bet_reg_znormed.nii.gz /media/sc-mlsg/PYW98854602/20210507_G3_Glioma_autoseg_preprocessed/00_Tumor_masks_resampled/8089857/8089857_mask.nii.gz
74 8164554
/media/sc-mlsg/PYW98854602/20210507_G3_Glioma_autoseg_preprocessed/05_Stripped_znormalized/8164554/CT1_r2s_bet_reg_znormed.nii.gz /media/sc-mlsg/PYW98854602/20210507_G3_Glioma_autoseg_preprocessed/00_Tumor_masks_resampled/8164554/8164554_mask.nii.gz
/media/sc-mlsg/PYW98854602/20210507_G3_Glioma_autoseg_preprocessed/05_Stripped_znormalized/8164554/FLAIR_r2s_bet_reg_znormed.nii.gz /media/sc-mlsg/PYW98854602/20210507_G3_Glioma_autoseg_preprocessed/00_Tumor_masks_resampled/8164554/8164554_mask.nii.gz
/media/sc-mlsg/PYW98854602/20210507_G3_Glioma_autoseg_preprocessed/05_Stripped_znormalized/8164554/T1_r2s_bet_reg_znormed.nii.gz /media/sc-mlsg/PYW98854602/20210507_G3_Glioma_autoseg_preprocessed/00_Tumor_masks_resampled/8164554/8164554_m

In [16]:
error_dict

{'/media/sc-mlsg/PYW98854602/20210426_Pilocytic_processed_autoseg_preprocessed/05_Stripped_znormalized/2396904': ValueError('No labels found in this mask (i.e. nothing is segmented)!'),
 '/media/sc-mlsg/PYW98854602/20210426_Pilocytic_processed_autoseg_preprocessed/05_Stripped_znormalized/5973276': ValueError('No labels found in this mask (i.e. nothing is segmented)!')}